In [51]:
import cv2
import torch
import torchvision
from torchvision import transforms
import numpy as np

In [5]:
model = torchvision.models.detection.ssd300_vgg16(pretrained = True)

C:\Users\lenovo\anaconda3\envs\ultra\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\lenovo\anaconda3\envs\ultra\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to C:\Users\lenovo/.cache\torch\hub\checkpoints\ssd300_vgg16_coco-b556d3b4.pth


  0%|          | 0.00/136M [00:00<?, ?B/s]

In [6]:
print(model.eval())

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [47]:
classNames = ["background", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", 
              "truck", "boat", "traffic light", "fire hydrant", "street sign", "stop sign", 
              "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", 
              "bear", "zebra", "giraffe", "hat", "backpack", "umbrella", "shoe", "eye glasses", 
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", 
              "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
              "bottle", "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", 
              "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", 
              "cake", "chair", "couch", "potted plant", "bed", "mirror", "dining table", "window", 
              "desk", "toilet", "door", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", 
              "microwave", "oven", "toaster", "sink", "refrigerator", "blender", "book", "clock", 
              "vase", "scissors", "teddy bear", "hair drier", "toothbrush", "hair brush"]

In [48]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [50]:
cap = cv2.VideoCapture("1900-151662242_small.mp4")
output_video_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


confidence_threshold = 0.5
while True:
    ret, image = cap.read()
    if not ret:
        break

    image_tensor = transform(image).unsqueeze(0)
    
    with torch.no_grad():
        results = model(image_tensor)
    
    boxes = results[0]['boxes']
    scores = results[0]['scores']
    labels = results[0]['labels']
    
    keep = scores > confidence_threshold
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]
    
    keep = torchvision.ops.nms(boxes, scores, iou_threshold=0.5)
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]
    
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i].int().tolist()
        confidence = scores[i].item()
        class_idx = labels[i].item()

        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 255), 2)
        
        label = f"{confidence:.2f} {classNames[class_idx]}"
        cv2.putText(image, label, (x1, max(35, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    out.write(image)

cap.release()
out.release()
cv2.destroyAllWindows()
